In [1]:
import json
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import influxdb
import folium
from folium.plugins import MarkerCluster

In [2]:
with open("../../../secrets/lopy.json") as fh:
    client = influxdb.InfluxDBClient(**json.load(fh))

### Database

We can play around with the client, we can list available database for this credentials:

In [3]:
client.get_list_database()

[{'name': 'chirpstack_lopy4'}]

We can select a database to interact with it:

In [4]:
#client.switch_database('smartcampus_coverage')

In [5]:
client.query("SHOW MEASUREMENTS;")

ResultSet({'('measurements', None)': [{'name': 'device_frmpayload_data_ax'}, {'name': 'device_frmpayload_data_ay'}, {'name': 'device_frmpayload_data_az'}, {'name': 'device_frmpayload_data_hdop'}, {'name': 'device_frmpayload_data_height'}, {'name': 'device_frmpayload_data_lat'}, {'name': 'device_frmpayload_data_lon'}, {'name': 'device_frmpayload_data_pitch'}, {'name': 'device_frmpayload_data_roll'}, {'name': 'device_frmpayload_data_time'}, {'name': 'device_uplink'}]})

In [6]:
t1 = pd.Timestamp.utcnow()
t0 = t1.floor('1D') - pd.Timedelta('2D')
t0 = pd.Timestamp("2020-05-08")
t0 = pd.Timestamp("2020-05-20")

In [7]:
q = """
SELECT * FROM /device.*/ WHERE time >= '{}' AND "time" < '{}'
GROUP BY "device_name";
""".format(t0, t1.isoformat())
print(q)


SELECT * FROM /device.*/ WHERE time >= '2020-05-20 00:00:00' AND "time" < '2020-08-22T13:53:53.820477+00:00'
GROUP BY "device_name";



We query the Database:

In [8]:
res = client.query(q)

We received 3 series back:

In [9]:
res.keys()

[('device_frmpayload_data_ax', {'device_name': 'node-GPS001'}),
 ('device_frmpayload_data_ay', {'device_name': 'node-GPS001'}),
 ('device_frmpayload_data_az', {'device_name': 'node-GPS001'}),
 ('device_frmpayload_data_hdop', {'device_name': 'node-GPS001'}),
 ('device_frmpayload_data_height', {'device_name': 'node-GPS001'}),
 ('device_frmpayload_data_lat', {'device_name': 'node-GPS001'}),
 ('device_frmpayload_data_lon', {'device_name': 'node-GPS001'}),
 ('device_frmpayload_data_pitch', {'device_name': 'node-GPS001'}),
 ('device_frmpayload_data_roll', {'device_name': 'node-GPS001'}),
 ('device_frmpayload_data_time', {'device_name': 'node-GPS001'}),
 ('device_uplink', {'device_name': 'node-GPS001'})]

We write a function that converts a InfluxDB Query result into a DataFrame:

In [10]:
def to_frame(result):
    keys = result.keys()
    frames = []
    for serie in result.raw['series']:
        frame = pd.DataFrame(serie['values'], columns=serie['columns'])
        frame = frame.assign(**serie['tags'])
        frame['measurement'] = serie["name"]
        frames.append(frame)
    frames = pd.concat(frames)
    frames['time'] = pd.to_datetime(frames['time'])
    return frames

And we apply it to the query result we just get back:

In [11]:
x = to_frame(res)
x.sample(5)

,time,application_name,dev_eui,dr,f_cnt,f_port,frequency,rssi,snr,value,device_name,measurement
1539,2020-08-18 01:01:50.741000+00:00,location-tracker,70b3d54992dfb698,None,None,2,None,None,None,0.971436,node-GPS001,device_frmpayload_data_az
35,2020-08-16 14:18:50.160000+00:00,location-tracker,70b3d54992dfb698,None,None,2,None,None,None,-0.133911,node-GPS001,device_frmpayload_data_az
1403,2020-08-17 22:00:58.845000+00:00,location-tracker,70b3d54992dfb698,None,None,2,None,None,None,0.034180,node-GPS001,device_frmpayload_data_ax
160,2020-08-16 19:06:50.669000+00:00,location-tracker,70b3d54992dfb698,None,None,2,None,None,None,2.190000,node-GPS001,device_frmpayload_data_hdop
1871,2020-08-21 11:01:36.177000+00:00,location-tracker,70b3d54992dfb698,None,None,2,None,None,None,39695.000000,node-GPS001,device_frmpayload_data_time


In [12]:
x['measurement'] = x['measurement'].apply(lambda x: x.split('_')[-1])

In [13]:
q = x.measurement == 'uplink'
xup = x[q]
xval = x[~q]

In [14]:
dup = xup.set_index(['time', 'device_name'])[['frequency', 'rssi', 'snr']]

In [15]:
dval = xval.pivot_table(index=['time', 'device_name'], columns=['measurement'], values='value')

In [16]:
d = pd.concat([dup, dval], axis=1).drop('time', axis=1).reset_index()

In [17]:
d['frequency'] = d['frequency'].astype(int)/1e6

In [18]:
d = d.sort_values(['device_name', 'time'])
d['lon'] = d['lon'].ffill()
d['lat'] = d['lat'].ffill()

In [19]:
d = d.dropna(subset=['lat'])

In [20]:
d

,time,device_name,frequency,rssi,snr,ax,ay,az,hdop,height,lat,lon,pitch,roll
1,2020-08-16 12:44:08.712000+00:00,node-GPS001,867.3,-53,9.8,0.098389,0.018677,0.968872,2.70,832.900024,50.810783,4.310470,-1.056972,-5.780953
2,2020-08-16 12:45:21.643000+00:00,node-GPS001,868.5,-53,7.5,0.097778,0.020020,0.966431,2.21,95.400002,50.810532,4.318675,-1.180682,-5.777224
3,2020-08-16 12:46:35.128000+00:00,node-GPS001,868.1,-51,9.8,0.099854,0.018066,0.966431,1.31,101.700005,50.810703,4.319078,-1.065290,-5.898980
4,2020-08-16 12:47:48.692000+00:00,node-GPS001,868.3,-55,11,0.099731,0.019653,0.969604,1.32,96.500000,50.810555,4.318625,-1.154922,-5.958310
5,2020-08-16 12:49:01.649000+00:00,node-GPS001,868.3,-58,10.2,0.099487,0.020142,0.966431,2.29,99.400002,50.810799,4.318850,-1.187667,-5.877497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1943,2020-08-22 12:18:51.060000+00:00,node-GPS001,867.9,-45,9.8,0.025879,0.024414,0.973389,3.06,41.500000,50.810814,4.319853,-1.436256,-1.522930
1944,2020-08-22 12:39:06.721000+00:00,node-GPS001,867.3,-45,10.5,0.029175,0.020752,0.972412,3.42,54.200001,50.810776,4.319223,-1.221997,-1.718502
1945,2020-08-22 12:59:37.371000+00:00,node-GPS001,868.3,-43,9.8,0.025757,0.024414,0.972046,NaN,NaN,50.810776,4.319223,-1.438243,-1.517843
1946,2020-08-22 13:20:06.478000+00:00,node-GPS001,867.5,-45,8,0.026001,0.022095,0.972778,NaN,NaN,50.810776,4.319223,-1.300672,-1.531070


In [21]:
p = [50.810532, 4.318675]

In [22]:
m = folium.Map(location=p, zoom_start=14)

In [23]:
k = folium.Marker(p, icon=folium.Icon(icon='glyphicon-download', color='red'),
                  popup='LoRaWAN Antenna').add_to(m)

In [24]:
def html(x, keys=[]):
    s = "<table>"
    s += "<caption>{}</caption>".format(x["device_name"])
    for k in keys:
        s += "<tr><th scope='row' align='right'>{key:}:</th><td>{value:}</td></tr>".format(key=k, value=x[k])
    s += "</table>"
    return s

In [25]:
mc = MarkerCluster()
for row in d.to_dict(orient='records'):
    mc.add_child(
        folium.Marker(
            location=[row['lat'],  row['lon']],
            icon=folium.Icon(icon='glyphicon-phone'),
            tooltip=folium.features.Tooltip(html(row, ['frequency', 'rssi', 'snr', 'hdop']))
        )
    )
m.add_child(mc)

In [26]:
m.save('lora.html')